In [1]:
import os
import sys
import matplotlib.pyplot as plt
import datetime as dt
import pandas as pd
import numpy as np
import julian
from numpy import floor

In [2]:
%load_ext julia.magic
%julia using Dates;
%julia using LinearAlgebra;
%julia using DataFrames;
%julia using SatelliteToolbox

Initializing Julia interpreter. This may take some time...


In [3]:
filepath = 'C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\MURI Project_1\\Data\\Temp\\'
os.chdir(filepath)
filename = 'CHAMP_NEUTRAL_08_2005.csv';
CHAMP_Data = pd.read_csv(filename)
CHAMP_Data.index = pd.DatetimeIndex(CHAMP_Data['Unnamed: 0'])
CHAMP_Data = CHAMP_Data.drop('Unnamed: 0',axis=1)

In [4]:
## Quiet-time background data using JB08
## Import Solar Indices

# F10, S10, M10, Y10 data release 5_4g (28-Jul-2021 13:38:06.00) by Space Environment Technologies
filepath = "C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\MURI Project_1\\Data\\Solar Indices\\"
os.chdir(filepath)

SOLdata = pd.read_fwf('SOLFSMY.txt',skiprows=4,infer_nrows = 1000,header = None)
SOLdata.columns = ['YYYY' ,'DDD','JulianDay','F10','F81c','S10','S81c','M10','M81c','Y10','Y81c','Ssrc']
DTCdata = np.array(pd.read_fwf('DTCFILE.txt',infer_nrows = 1000,header = None))


In [6]:
JD = [];F10 = [];S10 = [];M10 = []; Y10 = []
F10c = [];S10c = [];M10c = []; Y10c = [];DstDTC = [];
ap = [];F10a = [];
year = 2005; month = 8; day = 23; ##quiet day

for i in range(len(CHAMP_Data)):
    t0 = CHAMP_Data.index[i]
    # t1 = dt.datetime(year,month,day,t0.hour,t0.minute,t0.second)
    t1 = t0
    jd = julian.to_jd(t1)
    JD.append(jd)
    DOY = (t1 - dt.datetime(year,1,1,0,0,0))/dt.timedelta(days = 1) + 1
    j = np.where((SOLdata['YYYY'] == year) & (SOLdata['DDD'] == floor(DOY)))[0][0]
    F10.append(SOLdata['F10'][j-1])
    S10.append(SOLdata['S10'][j-1])
    M10.append(SOLdata['M10'][j-2])
    Y10.append(SOLdata['Y10'][j-5])
    F10c.append(SOLdata['F81c'][j-1]) 
    S10c.append(SOLdata['S81c'][j-1])
    M10c.append(SOLdata['M81c'][j-2])
    Y10c.append(SOLdata['Y81c'][j-5])
    j = np.where((DTCdata[:,1] == year) & (DTCdata[:,2] == floor(DOY)))[0][0]
    DstDTC.append(DTCdata[j,t1.hour+3])

In [7]:
%julia JD = $JD; F10 = $F10; S10 = $S10; M10 = $M10; Y10 = $Y10; F10c = $F10c; S10c = $S10c; M10c = $M10c; Y10c = $Y10c; DstDTC = $DstDTC
%julia LAT = py"CHAMP_Data['Latitude (deg)']"; LON = py"CHAMP_Data['Longitude (deg)']"; ALT = py"CHAMP_Data['Altitude (km)']";
%julia gc_lat = []; gc_alt = [];
%julia for i = 1:length(LAT); y,z = geodetic_to_geocentric(deg2rad(LAT[i]),ALT[i]*1000);y0,z0 = geodetic_to_geocentric(deg2rad(LAT[i]),0) ;push!(gc_lat,y); push!(gc_alt,z-z0); end; 
%julia global DataJB = []; 
rho = []; nO2 = []; nN2 = []; nO = []; nAr = []; nHe = []; nH = []; T_exo = [];Tz = [];
%julia for i= 1:length(JD); push!(DataJB,jb2008(JD[i],gc_lat[i],deg2rad(LON[i]),gc_alt[i],F10[i],F10c[i],S10[i],S10c[i],M10[i],M10c[i],Y10[i],Y10c[i],DstDTC[i])); end;

DataJB = %julia DataJB
for i in range(len(CHAMP_Data)):
    rho.append(DataJB[i].rho)
    nO2.append(DataJB[i].nO2)
    nN2.append(DataJB[i].nN2)
    nO.append(DataJB[i].nO)
    nAr.append(DataJB[i].nAr)
    nHe.append(DataJB[i].nHe)
    nH.append(DataJB[i].nH)
    T_exo.append(DataJB[i].T_exo)
    Tz.append(DataJB[i].Tz)
    
Data_JB = pd.DataFrame({'rho': rho,
                       'nO2': nO2,
                        'nN2': nN2,
                        'nO' : nO,
                        'nAr':nAr,
                        'nHe':nHe,
                        'nH':nH,
                        'T_exo':T_exo,
                        'Tz':Tz
                       },index = CHAMP_Data.index)

In [8]:
filepath = "C:\\Users\\soumy\\OneDrive - The University of Texas at Dallas\\Documents\\MURI Project\\Data\\Temp\\"
os.chdir(filepath)

filename = 'JB_output_08_2005.csv'
Data_JB.to_csv(filename)